### r estimation using

In [7]:
using NPZ
using PyPlot, Statistics, Distributions
using BenchmarkTools
include("../../src/function/save_TNT_TN.jl")
#include("../../covariance_matrix/code/smoothing_cov_mat_calc_truncate_qu.jl")
# calculate covariance matrix
nside = 4
lmin = 2
lmax = 2nside
#lmax = 8
sphenical_harmonics = set_sphenical_harmonics_terms!()

#==
cl_2048 = npzread("/Users/ikumakiyoshi/Library/Mobile Documents/com~apple~CloudDocs/study_fg_rm/program/julia_Delta_map/covariance_matrix/code/cl_th_nside_2048.npz")
cl_scal = cl_2048["cls_th"]
cl_lens = cl_2048["clL_th"]
cl_tens = cl_2048["clt_th"];

cl_ee_scal = cl_scal[:, 2] + cl_lens[:, 2]
cl_ee_tens = cl_tens[:, 2]
cl_bb_scal = cl_scal[:, 3] + cl_lens[:, 3]
cl_bb_tens = cl_tens[:, 3];
==#

#cov_mat_scal = calc_cmb_cov_mat(cl_ee_scal, cl_bb_scal, nside, lmin, lmax, sphenical_harmonics)
#cov_mat_tens = calc_cmb_cov_mat(cl_ee_tens, cl_bb_tens, nside, lmin, lmax, sphenical_harmonics)
#cov_mat_scal = npzread("../../covariance_matrix/smoothing_cov_mat_scal_nside_$(nside)_lmin_$(lmin)_lmax_$(lmax).npy")
#cov_mat_tens = npzread("../../covariance_matrix/smoothing_cov_mat_tens_nside_$(nside)_lmin_$(lmin)_lmax_$(lmax).npy")
#cov_mat_scal = npzread("../../covariance_matrix/smoothing_cov_mat_scal_nside_$(nside)_lmin_$(lmin)_lmax_$(lmax)_40GHz_beam.npy")
#cov_mat_tens = npzread("../../covariance_matrix/smoothing_cov_mat_tens_nside_$(nside)_lmin_$(lmin)_lmax_$(lmax)_40GHz_beam.npy")
#cov_mat_scal = npzread("../../covariance_matrix/smoothing_cov_mat_scal_nside_$(nside)_lmin_$(lmin)_lmax_$(lmax)_40GHz_beam_wo_wl.npy")
#cov_mat_tens = npzread("../../covariance_matrix/smoothing_cov_mat_tens_nside_$(nside)_lmin_$(lmin)_lmax_$(lmax)_40GHz_beam_wo_wl.npy")
#cov_mat_scal = npzread("../../covariance_matrix/smoothing_cov_mat_scal_nside_$(nside)_lmin_$(lmin)_lmax_$(lmax)_pixsize25_beam.npy")
#cov_mat_tens = npzread("../../covariance_matrix/smoothing_cov_mat_tens_nside_$(nside)_lmin_$(lmin)_lmax_$(lmax)_pixsize25_beam.npy")
#cov_mat_scal = npzread("../../covariance_matrix/paper_smoothing_cov_mat_scal_nside_$(nside)_lmin_$(lmin)_lmax_$(lmax)_pixsize25_beam_wo_wl.npy")
#cov_mat_tens = npzread("../../covariance_matrix/paper_smoothing_cov_mat_tens_nside_$(nside)_lmin_$(lmin)_lmax_$(lmax)_pixsize25_beam_wo_wl.npy")
cov_mat_scal = npzread("../../covariance_matrix/paper_smoothing_cov_mat_scal_nside_$(nside)_lmin_$(lmin)_lmax_$(lmax)_wo_beam_wo_wl.npy")
cov_mat_tens = npzread("../../covariance_matrix/paper_smoothing_cov_mat_tens_nside_$(nside)_lmin_$(lmin)_lmax_$(lmax)_wo_beam_wo_wl.npy")

# set parameters
#freq_bands = [40, 60, 140]
#freq_bands = [40, 50, 60, 68, 78, 89, 100, 119, 140]
#freq_bands = [140, 235, 280, 402]
#freq_bands = [100, 119, 140, 166, 195, 235, 280, 337, 402];
#freq_bands = [40, 60, 140, 235, 280, 337]
freq_bands = [40, 50, 60, 68, 78, 89, 100, 119, 140, 166, 195, 235, 280, 337, 402];
which_model = "d1 and s1"
r_input = 0.01
seed = 101
num_I = 2  
spin = 2 # spin-2 for Q, U map
lmax_alm = lmax

WX_data = npzread("../../WX_matrix/WXmat_spin_$(spin)_nside_$(nside)_lmax_$(lmax_alm).npz")

Wmat = WX_data["Wmat"]
Xmat = WX_data["Xmat"];

mask_path = "../../P06_nside_$nside.fits"
mask = hp.read_map(mask_path);
N⁻¹_set = Matrix{Float64}[]  
TᵀN⁻¹_set = Matrix{ComplexF64}[]  
TᵀN⁻¹T_set = Matrix{ComplexF64}[]  
m_set = Vector{Float64}[]  
r_est = 0.5

# definite SetParams and FitParams struct
set_params = SetParams(freq_bands, which_model, r_input, seed, nside, num_I, cov_mat_scal, cov_mat_tens, mask, m_set, N⁻¹_set, TᵀN⁻¹_set, TᵀN⁻¹T_set, spin, lmax_alm);
fit_params = FitParams(-3, 1.5, 20.1, r_est);
set_T0_matrix(set_params, sphenical_harmonics, Wmat, Xmat, mask_path);

In [8]:
sphenical_harmonics.T0;

In [9]:
#cov_mat_tens - cov_mat_tens_new

In [10]:
size(Wmat)

(192, 77)

In [11]:
set_truncate_N⁻¹!(set_params, lmin, lmax)[1]

214×214 Matrix{Float64}:
       5.9664e6         1.87342e6  …    73182.6             -1.38243e5
       1.87342e6        7.47813e6          -1.53594e5        1.18095e5
   66877.9              4.56383e6          -2.41016e5   -60118.0
       3.09867e6        1.04853e6      -84277.0          68737.7
      -4.45384e6       -5.31759e5          -2.46744e5        2.64025e5
      -4.3678e5        -3.64858e6  …       -2.1548e5        -1.29764e5
      -2.33893e5       -1.65656e6          -3.04245e5       -1.19469e5
       4.09687e5       -1.46541e5          -1.65821e5    -8472.85
       7.25485e5       -1.24593e6           4.29782e5        2.43753e5
       1.5104e6        -4.46412e5       33868.1              2.253e5
      -4.40222e5        6.82782e5  …        3.40406e5  -214357.0
       3.5061e5         2.0836e6      -352568.0         -24823.1
      -1.10005e5   449287.0             51669.9              4.7403e5
       ⋮                           ⋱                   
   65120.7              3.06

In [12]:
# save TᵀN⁻¹T
calc_TᵀN⁻¹T_terms(set_params::SetParams, mask_path, nside, lmin, lmax_alm)

In [43]:
using Healpix

function fwhm_pix_arcmin(nside::Int)
    θ_pix_rad = hp.nside2resol(nside)              
    return θ_pix_rad * 180 / π * 60                
end

delta = fwhm_pix_arcmin(nside) * 2.5

1099.35565178157